###OBJECTIF :

Nous avons une base de données de taille assez importante (lignes), qui contienne plusieurs variables (13). Exemple : signup_time : "le temps d'inscription", purchase_time : "le temps d'achats"...etc.

Le but de ce projet est de détecter les actions frauduleuses qui sont représentées dans la colonne de la variable catégorielle " class" : fraude / Pas_de_fraude.

Pour s'y entreprendre, il faut d'abord faire une analyse exploratoire des données, pour connaître par exemple les distributions de notre population selon l'âge, sex, pays... 

Ensuite, faire un preprocessing des données, à savoir, gérer les valeurs manquantes, transformer les données qualitatives à des données numériques à travers une dummification et, finalement normaliser les variables quantitatives afin d'avoir la même distribution des données.

L'étape suivante consiste à concevoir des modèles qui doivent être capable de repérer à l'avance les activités frauduleuses afin de les bloquer et par conséquent réduire le taux des activités frauduleuses.
Dans cette perspective, nous allons commencer avec un modèle de régression logistique, dans l’objectif de prédire le mieux possible les fraudes, et en fonction des scores, nous essayerons d’autres modèles jusqu’à trouver celui ou ceux qui généraliseront le mieux.  

### Etapes :
#### 1_ Analyse exploratoire
#### 2_ Preprocessiong & Feature engineering
#### 3_ Modélisation



In [0]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [0]:
path_1 = "https://raw.githubusercontent.com/JedhaBootcamp/FULL_STACK_12_WEEK_PROGRAM/master/S0.4%20--%20R%C3%A9duction%20de%20Dimensions%20%26%20Boosting/Projet%20--%20FRAUDULENT%20ACTIVITIES/Datasets/Fraud_Data.csv?token=AMEQSCMM7SWLNMYD233OA7S5HCGBM"

In [0]:
path_2 = "https://raw.githubusercontent.com/JedhaBootcamp/FULL_STACK_12_WEEK_PROGRAM/master/S0.4%20--%20R%C3%A9duction%20de%20Dimensions%20%26%20Boosting/Projet%20--%20FRAUDULENT%20ACTIVITIES/Datasets/IpAddress_to_Country.csv?token=AMEQSCKTK3U3TBYZFWLVOJK5HCGEK"

In [0]:
df1 = pd.read_csv(path_1)

In [0]:
df2 = pd.read_csv(path_2)

In [0]:
df1.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [0]:
df1.shape

(151112, 11)

In [0]:
df2.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [0]:
df2.shape

(138846, 3)

1- Pour chacun des utilisateurs, déterminez le pays d'origine depuis son adresse IP



In [0]:
def find_country(ip_address):
  countries = df2[(df2.lower_bound_ip_address <= ip_address) & (df2.upper_bound_ip_address > ip_address)].country #countries est une variable temporaire
  if len(countries) == 0:  
    return 'Unknown'
  return countries.values[0]

In [0]:
df1['country'] = df1.ip_address.apply(find_country)
df1.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,Unknown
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States


In [0]:
df1.to_csv('df1_country.csv')  # cela nous permettra d'aller chercher directement le dernier tableau df1 sans patienter longtemps la création du tableau à chaque utilisation

In [0]:
df1.describe(include='all')

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
count,151112.000000,151112,151112,151112.000000,151112,151112,151112,151112,151112.000000,1.511120e+05,151112.000000,151112
unique,NaN,151112,150679,NaN,137956,3,5,2,NaN,NaN,NaN,182
top,NaN,2015-06-13 04:34:50,2015-06-08 09:42:04,NaN,KIPFSCNUGOLDP,SEO,Chrome,M,NaN,NaN,NaN,United States
freq,NaN,1,3,NaN,20,60615,61432,88293,NaN,NaN,NaN,58049
mean,200171.040970,NaN,NaN,36.935372,NaN,NaN,NaN,NaN,33.140704,2.152145e+09,0.093646,NaN
std,115369.285024,NaN,NaN,18.322762,NaN,NaN,NaN,NaN,8.617733,1.248497e+09,0.291336,NaN
min,2.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,18.000000,5.209350e+04,0.000000,NaN
25%,100642.500000,NaN,NaN,22.000000,NaN,NaN,NaN,NaN,27.000000,1.085934e+09,0.000000,NaN
50%,199958.000000,NaN,NaN,35.000000,NaN,NaN,NaN,NaN,33.000000,2.154770e+09,0.000000,NaN
75%,300054.000000,NaN,NaN,49.000000,NaN,NaN,NaN,NaN,39.000000,3.243258e+09,0.000000,NaN


In [0]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 12 columns):
user_id           151112 non-null int64
signup_time       151112 non-null object
purchase_time     151112 non-null object
purchase_value    151112 non-null int64
device_id         151112 non-null object
source            151112 non-null object
browser           151112 non-null object
sex               151112 non-null object
age               151112 non-null int64
ip_address        151112 non-null float64
class             151112 non-null int64
country           151112 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 13.8+ MB
